In [120]:
#math
import numpy as np
import scipy.stats as stats
import scipy.special
#graphing
import matplotlib.pyplot as plt
#stats
import statsmodels.api as sm
from statsmodels.base.model import GenericLikelihoodModel

In [121]:
num_bidders = 4

def gen_data():
    nobs = 5000
    
    #parameters
    beta0 = 3
    beta1 = 3
    beta2 = 2
    sigma = 1
    
    gamma0 = 3
    gamma1 = 3 
    eta = 1
    
    epsilon = stats.norm.rvs(loc=0,scale=sigma,size=nobs)
    xi = stats.norm.rvs(loc=0,scale=eta,size=nobs)
    
    #censor data below x<0?
    z = stats.norm.rvs(loc=0,scale=sigma,size=nobs)
    x = gamma0 + gamma1*z + xi
    
    y = beta0+ beta1*x + beta2*z + epsilon
    
    #censor
    x[x<=0] = 0
    
    return y,np.concatenate([[z],[x]]).transpose(),nobs


yn,xn,nobs = gen_data()
print(xn.shape)
print(sm.add_constant(xn).shape)
print(scipy.stats.mode(sm.add_constant(xn)))

(5000, 2)
(5000, 3)
ModeResult(mode=array([[ 1.        , -3.61543611,  0.        ]]), count=array([[5000,    1,  894]]))


In [122]:
np.random.seed()
yn,xn,nobs = gen_data()

In [123]:
class MARregression(GenericLikelihoodModel):
    
    def __init__(self, *args, **kwargs):
        super(MARregression,self).__init__(*args,**kwargs)
        #print(self.fixed_params)
        #['beta'+str(i) for i in range(0,self.exog.shape[1]-1)] +
        names = (['sigma'] + 
                 ['gamma'+str(i) for i in range(0,self.exog.shape[1]-1)] + 
                 ['eta'])
        self._set_extra_params_names(names)
        self.start_params = np.array([1]* (2*self.exog.shape[1]+1)) 
        #self.start_params = np.array( range(1, (2*self.exog.shape[1]+2)))
        #2 sets of params for z, 1 for x, 2 variances...
        
        
    
    def loglikeobs(self, params):
        y = self.endog
        x = self.exog[:,-1]
        m = 1*(self.exog[:,-1] == 0) #missingness
        z = self.exog[:,0:-1]
        
        betaz = params[0:self.exog.shape[1]-1]
        betax = params[self.exog.shape[1]-1]
        sigma2 = max(params[self.exog.shape[1]],1e-3)
        gamma = params[self.exog.shape[1]+1:-1]
        eta2 = max(params[-1],1e-3)
        
        a2 = max(betax**2 *eta2 + sigma2,1e-3)
        b = gamma*betax + betaz
        
        c2 = max(eta2 *sigma2/(betax**2*eta2 + sigma2),1e-3)
        d = (betax*eta2)/(betax**2*eta2+sigma2)
        e = gamma - (betax*eta2) * (betaz + gamma*betax)/(betax**2 * eta2 + sigma2) 
        
        mu_x = y*d + np.matmul(z,e)
        mu_y = np.matmul(z,b)
        
        pr_y = stats.norm.logpdf( y, loc = mu_y, scale=np.sqrt(a2))
        pr_x = stats.norm.logpdf( x, loc = mu_x, scale=np.sqrt(c2))
        pr_m = stats.norm.logcdf( x, loc = mu_x, scale=np.sqrt(c2))

        ll = pr_y + (1-m)*pr_x + m*pr_m
    #- .5*np.log(a2) - a2
    #- .5*np.log(c2) - c2
        print(ll.sum(),params)
        return ll
    


model =  MARregression(yn,sm.add_constant(xn))
model_fit = model.fit(disp=False)
model_fit.summary()

-330096.0965376737 [1. 1. 1. 1. 1. 1. 1.]
-328039.3444282069 [1.05 1.   1.   1.   1.   1.   1.  ]
-328378.9425556731 [1.   1.05 1.   1.   1.   1.   1.  ]
-318231.5935152711 [1.   1.   1.05 1.   1.   1.   1.  ]
-316118.1715118849 [1.   1.   1.   1.05 1.   1.   1.  ]
-329634.4191284864 [1.   1.   1.   1.   1.05 1.   1.  ]
-329512.74672145455 [1.   1.   1.   1.   1.   1.05 1.  ]
-328995.03569913685 [1.   1.   1.   1.   1.   1.   1.05]
-320914.0803880816 [1.01428571 1.01428571 1.01428571 1.01428571 1.01428571 1.01428571
 1.01428571]
-318939.1062752184 [1.01836735 1.01836735 1.01836735 1.01836735 0.95408163 1.01836735
 1.01836735]
-316114.49526038027 [1.02361516 1.02361516 1.02361516 1.02361516 0.9909621  0.95932945
 1.02361516]
-309693.0063987593 [1.03542274 1.03542274 1.03542274 1.03542274 0.98644315 0.91399417
 1.03542274]
-311345.0993669692 [1.03373594 1.03373594 1.03373594 1.03373594 0.98708871 0.98475635
 0.96945023]
-307148.09946644923 [1.04337478 0.97908907 1.04337478 1.04337478 0.9

-46260.15348148197 [ 2.90465968  0.31614215  3.22274378  2.09887263 -0.32916845 -0.82137521
  2.20718592]
-47091.7029339173 [ 2.75438166  0.33502376  3.02642969  2.00206815 -0.27087732 -0.7229214
  2.14537387]
-47926.72945229248 [ 2.65390145  0.3324199   2.86164347  1.87622311 -0.23069123 -0.38017085
  2.06783446]
-45574.807840753885 [ 2.94217051  0.1128928   3.2471303   2.23310345 -0.28951465 -0.85656547
  2.2921867 ]
-45111.46038616421 [ 3.29475943 -0.13219749  3.47229089  2.31005797 -0.59521339 -0.79215337
  2.64972966]
-46309.72856171095 [ 3.21185147  0.16518319  3.41203886  2.08400738 -0.77114845 -0.72604378
  2.52824741]
-44377.66306333186 [ 3.35184301  0.18230365  3.34923977  2.01436648 -0.97828885 -0.51716525
  2.71269771]
-47622.348288693065 [ 3.84554992  0.10658936  3.52417915  1.88928704 -1.6539055  -0.20785936
  3.22524865]
-40975.924113984925 [ 3.30846268  0.08459301  3.19739412  1.83541495 -0.99988041 -0.15385223
  2.80882867]
-39567.85438240171 [ 3.65389664 -0.06114977  

-32764.672790162756 [ 4.09344554 -4.064469    3.5758531   3.56922499  1.47716358  0.60434727
  4.28480943]
-32548.164666540062 [ 3.94925782 -3.16533523  3.35105047  3.00224362  0.77887487  0.78382579
  4.09120088]
-32754.311668601767 [ 3.96706716 -3.0581191   3.2730306   2.81560583  0.6687343   1.01322479
  4.08142273]
-32572.541148272765 [ 4.09985464 -3.32513915  3.50471966  3.19619023  0.76910684  0.65240017
  4.23199086]
-32377.64781594727 [ 3.96454902 -3.55220385  3.46248682  3.40888123  1.27504342  0.51837814
  4.11069865]
-32428.65513569387 [ 3.95916724 -3.79966172  3.48398974  3.6191221   1.65138337  0.44715751
  4.09430445]
-32674.69836616717 [ 3.85282251 -3.54636167  3.37086132  3.43842794  1.32510075  0.51637257
  4.07202764]
-32473.038732268054 [ 3.84782128 -3.34680736  3.48889934  3.42852219  1.29149914  0.30180368
  3.9058925 ]
-32458.73216676514 [ 4.01631259 -3.86160264  3.44287289  3.53181649  1.52143669  0.65142342
  4.17839232]
-32411.255967426063 [ 4.11660122 -3.56911

-16567.74250905851 [5.78440624 3.72996548 2.35798814 2.1250086  2.563643   3.12698473
 0.64703241]
-16050.85391937113 [5.74329923 2.99996353 2.39669304 2.44570808 2.95525118 3.0586798
 0.87587159]
-16878.780753580308 [5.85153506 3.86018511 2.28542476 2.24259611 2.99336318 3.23244043
 0.44871333]
-16048.331751000866 [5.72320668 2.99711265 2.41873056 2.38869561 2.79025511 3.02732262
 0.92926467]
-16380.07210817624 [5.86089953 2.99423189 2.42528146 2.2209702  2.58490591 3.28496823
 1.04268583]
-15981.44627398574 [5.72721454 3.33371219 2.36470578 2.3877876  2.93769131 3.02120879
 0.70457654]
-16444.650740343022 [5.97344289 3.70778093 2.29653962 2.15781752 2.76051507 3.43014423
 0.69144637]
-16014.982902418422 [5.69712703 3.0770025  2.41098504 2.39957087 2.85955573 2.98747009
 0.84044021]
-16309.376444091418 [5.66727955 2.86268281 2.45992191 2.56978575 2.96908056 2.84728619
 0.9132712 ]
-15994.693971693741 [5.81453236 3.39374512 2.3501662  2.24881738 2.78453165 3.20634911
 0.75822116]
-1616

-15858.52083457998 [5.69638091 3.30985676 2.3255636  2.24530679 2.80273488 3.112162
 0.76470597]
-15864.127207411406 [5.7153872  3.29170967 2.31987189 2.23063391 2.79819277 3.16054493
 0.78695949]
-15858.580545186953 [5.67996022 3.27590072 2.32896828 2.25441502 2.80849145 3.09167736
 0.77235216]
-15854.924291224404 [5.69145264 3.31064643 2.32277993 2.22781816 2.79060337 3.121042
 0.76893486]
-15855.56065702346 [5.69248335 3.3374329  2.3196375  2.21034197 2.77737289 3.12928936
 0.76211018]
-15859.509077559482 [5.6654219  3.31605832 2.31918002 2.23007675 2.79330511 3.08823624
 0.75323098]
-15861.49177135371 [5.67533947 3.32203796 2.31592149 2.24939183 2.83033807 3.09618176
 0.74075937]
-15857.14287394844 [5.68817561 3.28414654 2.3268509  2.23715417 2.79016443 3.11212482
 0.77986646]
-15857.741930350827 [5.66936325 3.28055142 2.32850669 2.24500081 2.78984325 3.0782652
 0.77541613]
-15858.866178020728 [5.70121911 3.26093842 2.3318874  2.25658475 2.80679405 3.11737339
 0.79157384]
-15862.33

...

-14817.763467515946 [4.8783641  3.33126955 2.48972526 1.1202347  2.93580693 2.99428512
 1.29256463]
-14815.781133505898 [4.94045468 3.26370588 2.4784627  1.26644121 2.98165425 3.04225082
 1.10333989]
-14789.647370315539 [4.93138865 3.39458154 2.46614069 1.20223817 2.98177489 3.01921945
 1.17345013]
-14771.490264459018 [4.88349829 3.40384347 2.47259395 1.15007739 2.99970159 3.00244035
 1.19510098]
-14812.218471204196 [4.86610309 3.2168359  2.51070344 1.12978802 3.01228571 3.09124301
 1.14857891]
-14776.18848231874 [4.89304767 3.3319141  2.48830462 1.19450634 2.94597675 3.00490973
 1.10755033]
-14795.418868739724 [4.81372416 3.15837891 2.53303937 1.14191937 2.95555858 2.98982336
 1.1808595 ]
-14777.57029516801 [4.83347026 3.23721224 2.50234632 1.10023716 2.96866658 3.04125327
 1.20017884]
-14761.838261373934 [4.81243106 3.2443423  2.51016152 1.11501345 3.0138657  3.00894305
 1.18351333]
-14751.628689883399 [4.74307576 3.20443825 2.52664571 1.06182771 3.05482342 2.99089491
 1.20150855]
-1

-14672.636595428909 [4.53970547 3.157337   2.56134473 0.95196637 3.02157404 2.90829713
 0.99591824]
-14673.328843247124 [4.54081393 3.15772246 2.5616574  0.95196637 3.02157404 2.90829713
 0.99591824]
-14673.492685296265 [4.54081393 3.15772246 2.56103207 0.95196637 3.02157404 2.90829713
 0.99591824]
-14672.461917839053 [4.53970547 3.15772246 2.5616574  0.95196637 3.02157404 2.90829713
 0.99591824]
-14672.637664431222 [4.53970547 3.15772246 2.56103207 0.95196637 3.02157404 2.90829713
 0.99591824]
-14673.328976673536 [4.54081393 3.15772246 2.56134473 0.95208258 3.02157404 2.90829713
 0.99591824]
-14673.482978794347 [4.54081393 3.15772246 2.56134473 0.95185017 3.02157404 2.90829713
 0.99591824]
-14672.468089251422 [4.53970547 3.15772246 2.56134473 0.95208258 3.02157404 2.90829713
 0.99591824]
-14672.62191995899 [4.53970547 3.15772246 2.56134473 0.95185017 3.02157404 2.90829713
 0.99591824]
-14673.485904752586 [4.54081393 3.15772246 2.56134473 0.95196637 3.02194289 2.90829713
 0.99591824]
-

-14673.120257670435 [4.5402597  3.15772246 2.56134473 0.95196637 3.0212052  2.90794211
 0.99591824]
-14673.02722020169 [4.5402597  3.15772246 2.56134473 0.95196637 3.02194289 2.90829713
 0.99603981]
-14673.082152951365 [4.5402597  3.15772246 2.56134473 0.95196637 3.02194289 2.90829713
 0.99579667]
-14672.868105898964 [4.5402597  3.15772246 2.56134473 0.95196637 3.0212052  2.90829713
 0.99603981]
-14672.923014409016 [4.5402597  3.15772246 2.56134473 0.95196637 3.0212052  2.90829713
 0.99579667]
-14672.527227644003 [4.5402597  3.15772246 2.56134473 0.95196637 3.02157404 2.90900716
 0.99591824]
-14672.974767823198 [4.5402597  3.15772246 2.56134473 0.95196637 3.02157404 2.90829713
 0.99591824]
-14672.974767823198 [4.5402597  3.15772246 2.56134473 0.95196637 3.02157404 2.90829713
 0.99591824]
-14673.424766818713 [4.5402597  3.15772246 2.56134473 0.95196637 3.02157404 2.9075871
 0.99591824]
-14672.72327420519 [4.5402597  3.15772246 2.56134473 0.95196637 3.02157404 2.90865215
 0.99603981]
-14

/home/erichschulman/anaconda3/lib/python3.7/site-packages/statsmodels/base/model.py:512: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  "Check mle_retvals", ConvergenceWarning)


<class 'statsmodels.iolib.summary.Summary'>
"""
                            MARregression Results                             
==============================================================================
Dep. Variable:                      y   Log-Likelihood:                -14673.
Model:                  MARregression   AIC:                         2.935e+04
Method:            Maximum Likelihood   BIC:                         2.937e+04
Date:                Mon, 23 Nov 2020                                         
Time:                        17:14:27                                         
No. Observations:                5000                                         
Df Residuals:                    4997                                         
Df Model:                           2                                         
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const          4.5403      0.057     80.267      0.000       4.429       4.651
x1             3.1577      0.051     61.380      0.000       3.057       3.259
x2             2.5613      0.017    150.715      0.000       2.528       2.595
sigma          0.9520      0.019     49.574      0.000       0.914       0.990
gamma0         3.0216      0.014    209.574      0.000       2.993       3.050
gamma1         2.9083      0.015    196.925      0.000       2.879       2.937
eta            0.9959      0.019     51.520      0.000       0.958       1.034
==============================================================================
"""

In [115]:
 e = params[0:self.exog.shape[1]-1]
        d = params[self.exog.shape[1]-1]
        c2 = max(params[self.exog.shape[1]],1e-3)
        b = params[self.exog.shape[1]+1:-1]
        a2 = max(params[-1],1e-3)

IndentationError: unexpected indent (<ipython-input-115-c6b677a28529>, line 2)

In [ ]:
 betaz = params[0:self.exog.shape[1]-1]
        betax = params[self.exog.shape[1]-1]
        sigma2 = max(params[self.exog.shape[1]],1e-3)
        gamma = params[self.exog.shape[1]+1:-1]
        eta2 = max(params[-1],1e-3)
        
        a2 = max(betax**2 *eta2 + sigma2,1e-3)
        b = gamma*betax + betaz
        
        c2 = max(eta2 *sigma2/(betax**2*eta2 + sigma2),1e-3)
        d = (betax*eta2)/(betax**2*eta2+sigma2)
        e = gamma - (betax*eta2) * (betaz + gamma*betax)/(betax**2 * eta2 + sigma2) 
        